# 📊 Baseline Model - Simple Forecasting Benchmark

Tạo baseline model đơn giản để benchmark và so sánh với quantile regression models phức tạp.

## Mục tiêu
- Tạo simple baseline models (naive, moving average, linear regression)
- So sánh performance với quantile models
- Hiểu được improvement từ feature engineering và advanced models

## Baseline Models
1. **Naive Forecast**: Dùng giá trị cuối cùng
2. **Moving Average**: Trung bình động của N periods
3. **Simple Linear Regression**: Linear trend
4. **Seasonal Naive**: Dùng giá trị cùng kỳ năm trước


In [15]:
# Setup
import sys
from pathlib import Path
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.linear_model import LinearRegression
from sklearn.metrics import mean_absolute_error, mean_squared_error
import plotly.graph_objects as go

# Add project root to path
project_root = Path().resolve().parent
if str(project_root) not in sys.path:
    sys.path.insert(0, str(project_root))

from src.config import setup_project_path, get_dataset_config, OUTPUT_FILES, setup_logging
from src.pipelines._03_model_training import prepare_data, load_data

setup_project_path()
setup_logging()

# Import display for Jupyter notebooks
try:
    from IPython.display import display
except ImportError:
    # Fallback if not in Jupyter
    display = print

# Get config
config = get_dataset_config()
print(f"Dataset: {config['name']}")
print(f"Target: {config['target_column']}")


Dataset: FreshRetailNet-50K
Target: sales_quantity


## 1. Load Data và Prepare Train/Test Split


In [16]:
# Load feature table
df = load_data(OUTPUT_FILES['master_feature_table'])
print(f"Feature table shape: {df.shape}")

# Prepare data với time-based split
X_train, X_test, y_train, y_test, features, cat_features = prepare_data(df, config)

print(f"Training set: {X_train.shape}")
print(f"Test set: {X_test.shape}")

# Get time column for time-series baselines
time_col = config['time_column']
target_col = config['target_column']

# Create train/test dataframes với time info
train_df = df.loc[X_train.index].copy()
test_df = df.loc[X_test.index].copy()

train_df['target'] = y_train.values
test_df['target'] = y_test.values

print(f"\nTrain time range: {train_df[time_col].min()} to {train_df[time_col].max()}")
print(f"Test time range: {test_df[time_col].min()} to {test_df[time_col].max()}")


2025-11-13 01:31:35,950 - src.pipelines._03_model_training - INFO - Loading data from: D:\datastorm\E-Grocery_Forecaster\data\3_processed\master_feature_table.parquet...
2025-11-13 01:31:35,951 - src.pipelines._03_model_training - ERROR - File not found: D:\datastorm\E-Grocery_Forecaster\data\3_processed\master_feature_table.parquet. Run _02_feature_enrichment.py first.


SystemExit: 1

## 2. Baseline Model 1: Naive Forecast

Dự đoán bằng giá trị cuối cùng của mỗi product-store combination.


In [ ]:
# Naive: Use last known value for each product-store
def naive_forecast(train_df, test_df, group_cols=None, target='target'):
    """Naive forecast: last value per group."""
    # Auto-detect group columns
    if group_cols is None:
        if 'PRODUCT_ID' in train_df.columns and 'STORE_ID' in train_df.columns:
            group_cols = ['PRODUCT_ID', 'STORE_ID']
        elif 'product_id' in train_df.columns and 'store_id' in train_df.columns:
            group_cols = ['product_id', 'store_id']
        else:
            raise ValueError("Cannot find PRODUCT_ID/STORE_ID columns")
    
    # Get last value for each product-store
    last_values = train_df.groupby(group_cols)[target].last().reset_index()
    last_values.columns = group_cols + ['naive_forecast']
    
    # Merge với test set
    test_with_forecast = test_df[group_cols + [target]].merge(
        last_values, on=group_cols, how='left'
    )
    
    # Fill NaN với mean (for new product-store combinations)
    test_with_forecast['naive_forecast'] = test_with_forecast['naive_forecast'].fillna(
        train_df[target].mean()
    )
    
    return test_with_forecast['naive_forecast'].values

naive_pred = naive_forecast(train_df, test_df, target='target')

# Evaluate
naive_mae = mean_absolute_error(y_test, naive_pred)
naive_rmse = np.sqrt(mean_squared_error(y_test, naive_pred))

print(f"Naive Forecast:")
print(f"  MAE: {naive_mae:.4f}")
print(f"  RMSE: {naive_rmse:.4f}")


## 3. Baseline Model 2: Moving Average

Dự đoán bằng trung bình động của N periods gần nhất.


In [ ]:
# Moving Average: Average of last N periods
def moving_average_forecast(train_df, test_df, window=4, 
                           group_cols=None, time_col=None, target='target'):
    """Moving average forecast: average of last N periods."""
    # Auto-detect columns
    if group_cols is None:
        if 'PRODUCT_ID' in train_df.columns and 'STORE_ID' in train_df.columns:
            group_cols = ['PRODUCT_ID', 'STORE_ID']
        elif 'product_id' in train_df.columns and 'store_id' in train_df.columns:
            group_cols = ['product_id', 'store_id']
        else:
            raise ValueError("Cannot find PRODUCT_ID/STORE_ID columns")
    
    if time_col is None:
        if 'WEEK_NO' in train_df.columns:
            time_col = 'WEEK_NO'
        elif 'hour_timestamp' in train_df.columns:
            time_col = 'hour_timestamp'
        elif 'time_column' in config:
            time_col = config['time_column']
        else:
            raise ValueError("Cannot find time column")
    
    # Calculate moving average for each group
    train_sorted = train_df.sort_values(group_cols + [time_col])
    train_sorted['ma'] = train_sorted.groupby(group_cols)[target].transform(
        lambda x: x.rolling(window=window, min_periods=1).mean()
    )
    
    # Get last MA value for each group
    last_ma = train_sorted.groupby(group_cols)['ma'].last().reset_index()
    last_ma.columns = group_cols + ['ma_forecast']
    
    # Merge với test set
    test_with_forecast = test_df[group_cols + [target]].merge(
        last_ma, on=group_cols, how='left'
    )
    
    # Fill NaN
    test_with_forecast['ma_forecast'] = test_with_forecast['ma_forecast'].fillna(
        train_df[target].mean()
    )
    
    return test_with_forecast['ma_forecast'].values

ma_pred = moving_average_forecast(train_df, test_df, window=4, target='target')

# Evaluate
ma_mae = mean_absolute_error(y_test, ma_pred)
ma_rmse = np.sqrt(mean_squared_error(y_test, ma_pred))

print(f"Moving Average (window=4):")
print(f"  MAE: {ma_mae:.4f}")
print(f"  RMSE: {ma_rmse:.4f}")


## 4. Baseline Model 3: Simple Linear Regression

Dự đoán bằng linear trend cho mỗi product-store.


In [ ]:
# Linear Regression: Fit linear trend per group
def linear_trend_forecast(train_df, test_df, 
                         group_cols=None, time_col=None, target='target'):
    """Linear trend forecast: fit linear regression per group."""
    # Auto-detect columns
    if group_cols is None:
        if 'PRODUCT_ID' in train_df.columns and 'STORE_ID' in train_df.columns:
            group_cols = ['PRODUCT_ID', 'STORE_ID']
        elif 'product_id' in train_df.columns and 'store_id' in train_df.columns:
            group_cols = ['product_id', 'store_id']
        else:
            raise ValueError("Cannot find PRODUCT_ID/STORE_ID columns")
    
    if time_col is None:
        if 'WEEK_NO' in train_df.columns:
            time_col = 'WEEK_NO'
        elif 'hour_timestamp' in train_df.columns:
            time_col = 'hour_timestamp'
        elif 'time_column' in config:
            time_col = config['time_column']
        else:
            raise ValueError("Cannot find time column")
    
    predictions = []
    
    for (prod, store), group_train in train_df.groupby(group_cols):
        # Prepare features
        X_train_group = group_train[[time_col]].values
        y_train_group = group_train[target].values
        
        # Fit linear model
        if len(X_train_group) > 1:
            model = LinearRegression()
            model.fit(X_train_group, y_train_group)
            
            # Predict for test set
            group_test = test_df[(test_df[group_cols[0]] == prod) & 
                                (test_df[group_cols[1]] == store)]
            if len(group_test) > 0:
                X_test_group = group_test[[time_col]].values
                pred = model.predict(X_test_group)
                predictions.extend(pred)
            else:
                # No test data for this group
                pass
        else:
            # Not enough data, use mean
            mean_val = y_train_group[0] if len(y_train_group) > 0 else train_df[target].mean()
            group_test = test_df[(test_df[group_cols[0]] == prod) & 
                                (test_df[group_cols[1]] == store)]
            if len(group_test) > 0:
                predictions.extend([mean_val] * len(group_test))
    
    # Handle groups not in training
    if len(predictions) < len(test_df):
        remaining = len(test_df) - len(predictions)
        predictions.extend([train_df[target].mean()] * remaining)
    
    return np.array(predictions[:len(test_df)])

linear_pred = linear_trend_forecast(train_df, test_df, target='target')

# Evaluate
linear_mae = mean_absolute_error(y_test, linear_pred)
linear_rmse = np.sqrt(mean_squared_error(y_test, linear_pred))

print(f"Linear Trend:")
print(f"  MAE: {linear_mae:.4f}")
print(f"  RMSE: {linear_rmse:.4f}")


## 5. Compare All Baselines


In [ ]:
# Compare all baselines
results = pd.DataFrame({
    'Model': ['Naive', 'Moving Average (4)', 'Linear Trend'],
    'MAE': [naive_mae, ma_mae, linear_mae],
    'RMSE': [naive_rmse, ma_rmse, linear_rmse]
})

print("=" * 70)
print("BASELINE MODEL COMPARISON")
print("=" * 70)
display(results)

# Find best baseline
best_idx = results['MAE'].idxmin()
print(f"\n✅ Best Baseline: {results.loc[best_idx, 'Model']}")
print(f"   MAE: {results.loc[best_idx, 'MAE']:.4f}")
print(f"   RMSE: {results.loc[best_idx, 'RMSE']:.4f}")

# Visualize comparison
fig = go.Figure()

models = results['Model'].tolist()
mae_values = results['MAE'].tolist()

fig.add_trace(go.Bar(
    x=models,
    y=mae_values,
    text=[f'{v:.4f}' for v in mae_values],
    textposition='auto',
    marker_color='lightblue'
))

fig.update_layout(
    title='Baseline Models Comparison (MAE)',
    xaxis_title='Model',
    yaxis_title='Mean Absolute Error',
    height=400
)

fig.show()


## 6. Compare với Quantile Models (nếu đã train)

So sánh baseline với advanced quantile models.


In [ ]:
# Load quantile model metrics nếu có
import json
metrics_file = OUTPUT_FILES['reports_dir'] / 'metrics' / 'quantile_model_metrics.json'

if metrics_file.exists():
    with open(metrics_file, 'r') as f:
        quantile_metrics = json.load(f)
    
    # Get Q50 (median) model MAE (approximate)
    q50_pinball = quantile_metrics.get('q50_pinball_loss', None)
    
    if q50_pinball and 'results' in locals() and 'best_idx' in locals():
        print("=" * 70)
        print("COMPARISON: Baseline vs Quantile Models")
        print("=" * 70)
        print(f"Best Baseline MAE: {results.loc[best_idx, 'MAE']:.4f}")
        print(f"Q50 Quantile Pinball Loss: {q50_pinball:.4f}")
        print(f"\nNote: Pinball loss và MAE không trực tiếp comparable,")
        print(f"      nhưng có thể thấy improvement từ feature engineering")
    elif q50_pinball:
        print("=" * 70)
        print("QUANTILE MODEL METRICS")
        print("=" * 70)
        print(f"Q50 Quantile Pinball Loss: {q50_pinball:.4f}")
        print("\nNote: Run baseline models above to compare")
else:
    print("Quantile models chưa được train.")
    print("Chạy notebook 03_Model_Training.ipynb để train quantile models trước.")
